In [ ]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title imports
%%capture
%cd /content/drive/MyDrive/DepthEstimation/
 
!pip install tensorflow-addons
# !pip install segmentation-models
 
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
 
import matplotlib.pyplot as plt
import os
from glob import glob 
import numpy as np
import seaborn as sns
import pandas as pd
import json
import tensorflow as tf
import tensorflow_addons as tfa
import ipywidgets as widgets
from tqdm.notebook import tqdm
import IPython

# internal codes
from tf_implementation.scripts.dataloaders import OrdinaryDataloader
from tf_implementation.scripts.models.ordinary_unet import OrdinaryUNet
from tf_implementation.scripts.models.efficient_unet import EfficientUNet
 
from tf_implementation.scripts.utils import good
from tf_implementation.scripts.utils import update_train_filenames_file
 
from tf_implementation.config import Config

In [ ]:
#@title version control
%cd /content/drive/MyDrive/DepthEstimation/
def create_version(_):
    if version_name_widget.value == '':
        print('please set a name for this version')
        return
    config.create_version(version_name_widget.value)
 
version_name_widget = widgets.Text(description='Version Name',
                                   value=None)
version_widget = widgets.Dropdown(options=glob('tf_implementation/ckpts/*'),
                                  description='Version',
                                  value=None)
create_version_widget = widgets.Button(description='Create New Version')
 
base_setting = json.load(open(('tf_implementation/settings/setting_kitti_base.json'),
                         'r'))
 
config = Config(base_setting)
 
create_version_widget.on_click(create_version)
version_widget.observe(lambda x:config.load_version(x['new']),
                       names='value')
display(version_widget,version_name_widget,create_version_widget)

In [ ]:
#@title training
%cd /content/drive/MyDrive/DepthEstimation/

early_stopping_counter = 0

train_dataloader = OrdinaryDataloader(config)
test_dataloader = OrdinaryDataloader(config,
                                     is_training=False)

train_summary_writer = tf.summary.create_file_writer(config.train_log_dir)
test_summary_writer = tf.summary.create_file_writer(config.test_log_dir)

dataloader_iter = iter(train_dataloader.loader)
if config.encoder.startswith('DenseNet'):
    ounet = OrdinaryUNet(config)
else:
    ounet = EfficientUNet(config)

ckpt = tf.train.Checkpoint(step=tf.Variable(0),
                           train_loss=tf.Variable(0.0),
                           test_loss=tf.Variable(0.0),
                           strategy=tf.Variable(0),
                           optimizer = ounet.optimizer,
                           model=ounet.model)

last_manager = tf.train.CheckpointManager(ckpt,
                                          directory=config.last_ckpt_path,
                                          max_to_keep=config.max_to_keep_last_ckpt)
best_manager = tf.train.CheckpointManager(ckpt,
                                          directory=config.best_ckpt_path,
                                          max_to_keep=config.max_to_keep_best_ckpt)

if best_manager.latest_checkpoint:
    ckpt.restore(best_manager.latest_checkpoint)
    best_test_loss=ckpt.test_loss.numpy()
    print(f'best test loss set to {best_test_loss}')
else:
    print('best test loss set to inf')
    best_test_loss = np.inf

if last_manager.latest_checkpoint:
    ckpt.restore(last_manager.latest_checkpoint)
    step = ckpt.step.numpy()
    best_train_loss=ckpt.train_loss.numpy()
    train_dataloader.current_strategy = ckpt.strategy.numpy()
    print(f'restored from {last_manager.latest_checkpoint} starting from [{step-1}][{train_dataloader.current_strategy}]')
else:
    print('Initializing from scratch.')
    best_train_loss = np.inf
    step = 0

while step < config.max_steps:
    image, depth_gt = dataloader_iter.get_next()
    loss,amax,amin = ounet.train_step(image, depth_gt)

    with train_summary_writer.as_default():
        tf.summary.scalar('loss', loss , step=step)
        tf.summary.scalar('amax est', amax.numpy() , step=step)
        tf.summary.scalar('amin est', amin.numpy() , step=step)
        tf.summary.scalar('amax gt', np.amax(depth_gt) , step=step)
        tf.summary.scalar('amin gt', np.amin(depth_gt) , step=step)
        train_summary_writer.flush()

    if step > config.num_warmup_steps:
        if not step%config.num_steps_per_train_checkpoint and\
                config.save_checkpoints:
            ckpt.step.assign(step)
            ckpt.train_loss.assign(loss)
            ckpt.strategy.assign(train_dataloader.current_strategy)
            last_manager.save()
            print(f'Train checkpoint-[{step}][{train_dataloader.current_strategy}]')

        if  loss < config.early_stopping_thresh*best_train_loss:
            early_stopping_counter = 0
            best_train_loss = loss
        elif train_dataloader.current_strategy < train_dataloader.num_strategies-1:
            early_stopping_counter += 1
            if early_stopping_counter > train_dataloader.early_stopping_patience:
                train_dataloader.current_strategy += 1
                early_stopping_counter = 0

    if step > config.num_warmup_steps and\
                not step%config.num_steps_per_test_checkpoint and\
                config.save_checkpoints:
        
        metrics = dict(a1=[], a2=[], a3=[],
                    abs_rel=[], rmse=[], log_10=[],
                    rmse_log=[], silog=[], sq_rel=[])
        test_loss = []
        for test_image, test_depth_gt in tqdm(test_dataloader.loader):
            temp_metrics = ounet.compute_metrics(test_image,
                                             test_depth_gt)
            loss,depth_est = ounet.test_step(test_image,
                                              test_depth_gt)
            test_loss.append(loss.numpy())
            [metrics[k].append(v) for k,v in temp_metrics.items()]
        metrics = {k:np.mean(v) for k,v in metrics.items()}
        metrics['loss'] = np.mean(test_loss)
        print('metrics',
              ', '.join(f'{k} = {v}' for k,v in metrics.items()))
        with test_summary_writer.as_default():
            tf.summary.image('depth_est_sample',
                             depth_est,
                             step=step)
            tf.summary.image('gt_sample',
                             test_depth_gt,
                             step=step)
            tf.summary.image('input_sample',
                             test_image,
                             step=step)
            for name, value in metrics.items():
                tf.summary.scalar(name, value, step=step)
            test_summary_writer.flush()

        if  metrics['loss'] < best_test_loss:
            best_test_loss = metrics['loss']
            ckpt.step.assign(step)
            ckpt.test_loss.assign(best_test_loss)
            ckpt.strategy.assign(train_dataloader.current_strategy)
            ckpt.save_counter.assign_sub(1)
            best_manager.save()
            print(f'Test checkpoint-[{step}][{train_dataloader.current_strategy}]')

    print(f'train\t[{step}][{train_dataloader.current_strategy}][{early_stopping_counter}=<{train_dataloader.early_stopping_patience}]\tloss\t{loss:1.5f}')
    step+=1

In [ ]:
#@title remove checkpoints & logs
command = "list" #@param ["list", "remove"]
address = "ckpts" #@param ["logs", "ckpts"]
if command == "remove":
    if address == 'logs':
        !rm -rf tf_implementation/logs/*
    else:
        !rm -rf tf_implementation/ckpts/*
else:
    !ls tf_implementation/ckpts/best_models/*
    !ls tf_implementation/ckpts/last_model/*
    !ls tf_implementation/logs/*

# TfLite

In [ ]:
%cd /content/drive/MyDrive/DepthEstimation/

early_stopping_counter = 0

if config.encoder.startswith('DenseNet'):
    ounet = OrdinaryUNet(config)
else:
    ounet = EfficientUNet(config)

ckpt = tf.train.Checkpoint(step=tf.Variable(0),
                           train_loss=tf.Variable(0.0),
                           test_loss=tf.Variable(0.0),
                           strategy=tf.Variable(0),
                           optimizer = ounet.optimizer,
                           model=ounet.model)

best_manager = tf.train.CheckpointManager(ckpt,
                                          directory=config.best_ckpt_path,
                                          max_to_keep=config.max_to_keep_best_ckpt)

if best_manager.latest_checkpoint:
    ckpt.restore(best_manager.latest_checkpoint)
    best_test_loss=ckpt.test_loss.numpy()
    print(f'best test loss set to {best_test_loss}')
else:
    print('best test loss set to inf')
    best_test_loss = np.inf

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('final_model_pb/ounet.pb') 
tflite_model = converter.convert()

# Save the model.
with open('final_model_pb/ounet.tflite', 'wb') as f:
  f.write(tflite_model)